# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/ebaranauskas/repos/dend-c2-data-modeling-with-cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [20]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
# helper function for reading data and parsing numeric fields
def gen_parsed_data(file = 'event_datafile_new.csv'):
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.DictReader(f)
        for line in csvreader:
            line['userId'] = int(line['userId'])
            line['sessionId'] = int(line['sessionId'])
            line['itemInSession'] = int(line['itemInSession'])
            line['length'] = float(line['length'])
            
            yield line

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

# Helper function for executing queries against session
def try_exec_query(q, **kwargs):
    try:
        res = session.execute(q, kwargs)
        return res
    except Exception as e:
        print(e)
        raise

#### Create Keyspace

In [7]:
try_exec_query("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}    
""")

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('sparkifydb')
except Exception as ex:
    print(ex)
    raise

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Task 1 Solution

The query requires us to retrieve the artist, song title, and song’s length in the music app history that was heard during `sessionId = 338`, and `itemInSession = 4`. Fundamentally this query will be retrieving song information that was played on a specific item during a session, so the appropriate name for this table is `session_events`. From this query specification we can derive that we need to store `sessionId`, `itemInSession`, `artist`, `song`, and `length`. The composite primary key will be made up of `sessionId` and `itemInSession`, since this is the minimum information we need to make the records unique as well as these are also the fields that we will use to find an answer to our query.

In [9]:
session_events_create = """
    CREATE TABLE IF NOT EXISTS session_events (
        sessionId BIGINT,
        itemInSession INT,
        artist TEXT,
        song TEXT,
        length DOUBLE,        
        PRIMARY KEY (sessionId, itemInSession)
    )
"""
try_exec_query(session_events_create)             

In [10]:
session_events_insert = """
    INSERT INTO session_events (
        sessionId,
        itemInSession,
        artist,
        song,
        length
    ) VALUES (
        %(sessionId)s,
        %(itemInSession)s,
        %(artist)s,
        %(song)s,
        %(length)s          
    )
"""

for r in gen_parsed_data():        
    try_exec_query(session_events_insert, **r)

In [11]:
session_events_query = """
    SELECT
        artist, song, length
    FROM session_events
    WHERE sessionId = %(sessionId)s AND itemInSession = %(itemInSession)s
"""

rows = try_exec_query(
    session_events_query,
    sessionId=338,
    itemInSession=4
)

for r in rows:
    print(r)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.3073)


## Task 2 Solution

The query requires us to retrieve the name of artist, song (sorted by `itemInSession`) and user (first and last name) for `userId = 10`, `sessionId = 182`. Fundamentally this query will be retrieving song information that was played by a specific user during one of their sessions, so the appropriate name for this table is `user_events`. From this query specification we can derive that we need to store `userId`, `sessionId`, `itemInSession`, `artist`, `song`, `firstName` and `lastName`. The composite primary key will be made up of `userId` and `sessionId`, however we will also have `itemInSession` as a clustering column, since we need the query result to be sorted in the order songs were played during a user session.

In [12]:
user_events_create = """
    CREATE TABLE IF NOT EXISTS user_events (
        userId BIGINT,
        sessionId BIGINT,
        itemInSession INT,
        artist TEXT,
        song TEXT,
        firstName TEXT,
        lastName TEXT,
        PRIMARY KEY ((userId, sessionid), itemInSession)
    )
"""
try_exec_query(user_events_create) 

In [13]:
user_events_insert = """
    INSERT INTO user_events (
        userId,
        sessionId,
        itemInSession,
        artist,
        song,
        firstName,
        lastName
    ) VALUES (
        %(userId)s,
        %(sessionId)s,
        %(itemInSession)s,
        %(artist)s,
        %(song)s,
        %(firstName)s,
        %(lastName)s       
    )
"""

for r in gen_parsed_data():        
    try_exec_query(user_events_insert, **r)

In [14]:
user_events_query = """
    SELECT
        artist, song, firstName, lastName
    FROM user_events
    WHERE userId = %(userId)s AND sessionId = %(sessionId)s
"""

rows = try_exec_query(
    user_events_query,
    userId=10,
    sessionId=182
)

for r in rows:
    print(r)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


## Task 3 Solution

The query requires us to retrieve every user name (first and last) in the music app history who listened to the song ‘All Hands Against His Own’. Fundamentally this query will be retrieving user information who listened for a particular song, so the appropriate name for this table is `song_events`. From this query specification we can derive that we need to store `song`, `userId`, `firstName`, and `lastName`. The composite primary key will be made up of `song` and `userId`, singe `song` alone will not uniquely identify all users that played a particular song.

In [15]:
song_events_create = """
    CREATE TABLE IF NOT EXISTS song_events (
        song TEXT,
        userId BIGINT,
        firstName TEXT,
        lastName TEXT,        
        PRIMARY KEY (song, userId)
    )
"""
try_exec_query(song_events_create) 

In [16]:
song_events_insert = """
    INSERT INTO song_events (
        song,
        userId,
        firstName,
        lastName
    ) VALUES (
        %(song)s,
        %(userId)s,
        %(firstName)s,
        %(lastName)s     
    )
"""

for r in gen_parsed_data():        
    try_exec_query(song_events_insert, **r)

In [17]:
user_events_query = """
    SELECT
        firstName, lastName
    FROM song_events
    WHERE song = %(song)s
"""

rows = try_exec_query(
    user_events_query,
    song='All Hands Against His Own'
)

for r in rows:
    print(r)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [18]:
try_exec_query('DROP TABLE session_events')
try_exec_query('DROP TABLE user_events')
try_exec_query('DROP TABLE song_events')

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()